# Notebook 03 - ANÁLISE E VISUALIZAÇÃO
# Pipeline de Big Data - Análise de Churn Telecom
# Equipe: [NOMES DOS MEMBROS]

## 🎯 Objetivo
Este notebook implementa a etapa final do pipeline de Big Data, realizando **análise exploratória** e **visualizações** dos dados transformados para gerar insights acionáveis sobre churn de clientes.

## 📋 Etapas:
1. Carregamento dos dados transformados (Silver)
2. Análise exploratória de dados
3. Criação de visualizações
4. Geração de insights e métricas
5. Salvamento na camada Gold


In [ ]:
# ============================================================
# ETAPA 1: CONFIGURAÇÃO DO AMBIENTE
# ============================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

# Configurar visualizações
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Ambiente configurado com sucesso!")
print(f"📅 Data/Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("📊 Bibliotecas carregadas: pandas, numpy, matplotlib, seaborn")


In [ ]:
# ============================================================
# ETAPA 2: CARREGAMENTO DOS DADOS TRANSFORMADOS (SILVER)
# ============================================================

print("\n" + "="*60)
print("📥 CARREGANDO DADOS DA CAMADA SILVER")
print("="*60)

# Carregar dados transformados
caminho_silver = 'dados/silver/telco_churn_transformed.csv'

if os.path.exists(caminho_silver):
    df = pd.read_csv(caminho_silver)
    print(f"✅ Dados carregados com sucesso!")
    print(f"   📊 Registros: {len(df):,}")
    print(f"   📋 Colunas: {df.shape[1]}")
else:
    raise FileNotFoundError(f"❌ Arquivo não encontrado: {caminho_silver}")

# Verificar estrutura dos dados
print(f"\n📊 Informações gerais:")
print(f"   - Dimensões: {df.shape}")
print(f"   - Memória: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"   - Valores ausentes: {df.isnull().sum().sum()}")

# Verificar variável alvo
print(f"\n🎯 Distribuição da variável alvo:")
churn_dist = df['Churn_numeric'].value_counts()
print(f"   - Churn (1): {churn_dist[1]:,} ({churn_dist[1]/len(df)*100:.1f}%)")
print(f"   - Não Churn (0): {churn_dist[0]:,} ({churn_dist[0]/len(df)*100:.1f}%)")


In [ ]:
# ============================================================
# ETAPA 3: VISUALIZAÇÃO 1 - DISTRIBUIÇÃO GERAL DO CHURN
# ============================================================

print("\n" + "="*60)
print("📊 VISUALIZAÇÃO 1: DISTRIBUIÇÃO GERAL DO CHURN")
print("="*60)

# Criar figura com subplots
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Distribuição absoluta
churn_counts = df['Churn'].value_counts()
axes[0].pie(churn_counts.values, labels=churn_counts.index, autopct='%1.1f%%', 
           colors=['#ff9999', '#66b3ff'], startangle=90)
axes[0].set_title('Distribuição do Churn', fontsize=14, fontweight='bold')

# Gráfico 2: Distribuição com valores
sns.countplot(data=df, x='Churn', ax=axes[1], palette=['#ff9999', '#66b3ff'])
axes[1].set_title('Contagem de Churn', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Status do Cliente')
axes[1].set_ylabel('Número de Clientes')

# Adicionar valores nas barras
for i, v in enumerate(churn_counts.values):
    axes[1].text(i, v + 50, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('dados/gold/viz_01_distribuicao_churn.png', dpi=300, bbox_inches='tight')
plt.show()

# Estatísticas
taxa_churn = (df['Churn'] == 'Yes').mean() * 100
print(f"\n📈 Taxa geral de churn: {taxa_churn:.2f}%")
print(f"📊 Total de clientes: {len(df):,}")
print(f"🔴 Clientes que cancelaram: {(df['Churn'] == 'Yes').sum():,}")
print(f"🟢 Clientes ativos: {(df['Churn'] == 'No').sum():,}")


In [ ]:
# ============================================================
# ETAPA 4: VISUALIZAÇÃO 2 - CHURN POR TIPO DE CONTRATO
# ============================================================

print("\n" + "="*60)
print("📊 VISUALIZAÇÃO 2: CHURN POR TIPO DE CONTRATO")
print("="*60)

# Análise de churn por contrato
churn_por_contrato = df.groupby('Contract')['Churn_numeric'].agg(['count', 'sum', 'mean']).round(3)
churn_por_contrato.columns = ['Total_Clientes', 'Churn_Count', 'Taxa_Churn']
churn_por_contrato['Taxa_Churn_Pct'] = churn_por_contrato['Taxa_Churn'] * 100

print("📋 Resumo por tipo de contrato:")
print(churn_por_contrato)

# Visualização
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico 1: Taxa de churn por contrato
contract_order = ['Month-to-month', 'One year', 'Two year']
churn_rates = [churn_por_contrato.loc[contract, 'Taxa_Churn_Pct'] for contract in contract_order]

bars1 = axes[0].bar(contract_order, churn_rates, color=['#ff6b6b', '#4ecdc4', '#45b7d1'])
axes[0].set_title('Taxa de Churn por Tipo de Contrato', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Taxa de Churn (%)')
axes[0].set_xlabel('Tipo de Contrato')

# Adicionar valores nas barras
for i, v in enumerate(churn_rates):
    axes[0].text(i, v + 1, f'{v:.1f}%', ha='center', fontweight='bold')

# Gráfico 2: Distribuição de contratos
contract_counts = df['Contract'].value_counts()
axes[1].pie(contract_counts.values, labels=contract_counts.index, autopct='%1.1f%%',
           colors=['#ff6b6b', '#4ecdc4', '#45b7d1'], startangle=90)
axes[1].set_title('Distribuição de Contratos', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('dados/gold/viz_02_churn_por_contrato.png', dpi=300, bbox_inches='tight')
plt.show()

# Insights
print(f"\n💡 INSIGHTS:")
print(f"   🔴 Contratos mensais têm a MAIOR taxa de churn: {churn_rates[0]:.1f}%")
print(f"   🟡 Contratos anuais têm taxa INTERMEDIÁRIA: {churn_rates[1]:.1f}%")
print(f"   🟢 Contratos bienais têm a MENOR taxa de churn: {churn_rates[2]:.1f}%")
print(f"   📊 Diferença entre mensal e bienal: {churn_rates[0] - churn_rates[2]:.1f} pontos percentuais")


In [ ]:
# ============================================================
# ETAPA 5: VISUALIZAÇÃO 3 - ANÁLISE DE TENURE (PERMANÊNCIA)
# ============================================================

print("\n" + "="*60)
print("📊 VISUALIZAÇÃO 3: ANÁLISE DE TENURE (PERMANÊNCIA)")
print("="*60)

# Criar grupos de tenure
df['TenureGroup'] = pd.cut(df['tenure'], 
                          bins=[0, 12, 24, 36, 48, 60, 72], 
                          labels=['0-12m', '13-24m', '25-36m', '37-48m', '49-60m', '61-72m'])

# Análise de churn por grupo de tenure
churn_por_tenure = df.groupby('TenureGroup')['Churn_numeric'].agg(['count', 'sum', 'mean']).round(3)
churn_por_tenure.columns = ['Total_Clientes', 'Churn_Count', 'Taxa_Churn']
churn_por_tenure['Taxa_Churn_Pct'] = churn_por_tenure['Taxa_Churn'] * 100

print("📋 Resumo por grupo de permanência:")
print(churn_por_tenure)

# Visualização
fig, axes = plt.subplots(2, 1, figsize=(14, 12))

# Gráfico 1: Taxa de churn por tenure
tenure_groups = churn_por_tenure.index.tolist()
churn_rates_tenure = churn_por_tenure['Taxa_Churn_Pct'].tolist()

bars = axes[0].bar(tenure_groups, churn_rates_tenure, color='skyblue', edgecolor='navy', alpha=0.7)
axes[0].set_title('Taxa de Churn por Tempo de Permanência', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Taxa de Churn (%)')
axes[0].set_xlabel('Grupo de Permanência (meses)')

# Adicionar valores nas barras
for i, v in enumerate(churn_rates_tenure):
    axes[0].text(i, v + 0.5, f'{v:.1f}%', ha='center', fontweight='bold')

# Gráfico 2: Distribuição de tenure
axes[1].hist(df['tenure'], bins=20, color='lightcoral', edgecolor='darkred', alpha=0.7)
axes[1].set_title('Distribuição do Tempo de Permanência', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Tempo de Permanência (meses)')
axes[1].set_ylabel('Número de Clientes')

# Adicionar linha de média
media_tenure = df['tenure'].mean()
axes[1].axvline(media_tenure, color='red', linestyle='--', linewidth=2, 
                label=f'Média: {media_tenure:.1f} meses')
axes[1].legend()

plt.tight_layout()
plt.savefig('dados/gold/viz_03_analise_tenure.png', dpi=300, bbox_inches='tight')
plt.show()

# Insights
print(f"\n💡 INSIGHTS:")
print(f"   🔴 Clientes novos (0-12m) têm maior risco de churn")
print(f"   🟢 Clientes antigos (37m+) têm menor risco de churn")
print(f"   📊 Tempo médio de permanência: {media_tenure:.1f} meses")
print(f"   ⚠️ Primeiros 12 meses são críticos para retenção")


In [ ]:
# ============================================================
# ETAPA 6: SALVAMENTO NA CAMADA GOLD E RELATÓRIO FINAL
# ============================================================

print("\n" + "="*60)
print("💾 SALVAMENTO NA CAMADA GOLD")
print("="*60)

# Criar datasets agregados para análise
print("\n📊 Criando datasets agregados...")

# 1. Métricas por contrato
metricas_contrato = df.groupby('Contract').agg({
    'Churn_numeric': ['count', 'sum', 'mean'],
    'MonthlyCharges': 'mean',
    'TotalCharges': 'mean',
    'tenure': 'mean'
}).round(2)

metricas_contrato.columns = ['Total_Clientes', 'Churn_Count', 'Taxa_Churn', 
                            'Cobranca_Mensal_Media', 'Cobranca_Total_Media', 'Tenure_Medio']
metricas_contrato['Taxa_Churn_Pct'] = metricas_contrato['Taxa_Churn'] * 100

metricas_contrato.to_csv('dados/gold/metricas_por_contrato.csv')
print("   ✅ metricas_por_contrato.csv")

# 2. Perfil de clientes de alto risco
alto_risco = df[(df['Contract'] == 'Month-to-month') & (df['tenure'] <= 12)]
perfil_alto_risco = alto_risco.groupby(['Contract', 'TenureGroup']).agg({
    'Churn_numeric': ['count', 'sum', 'mean'],
    'MonthlyCharges': 'mean',
    'TotalServices': 'mean'
}).round(2)

perfil_alto_risco.columns = ['Total_Clientes', 'Churn_Count', 'Taxa_Churn', 
                            'Cobranca_Mensal_Media', 'Servicos_Medios']
perfil_alto_risco['Taxa_Churn_Pct'] = perfil_alto_risco['Taxa_Churn'] * 100

perfil_alto_risco.to_csv('dados/gold/perfil_alto_risco.csv')
print("   ✅ perfil_alto_risco.csv")

# 3. Análise por segmento de valor
segmento_valor = df.groupby('ValueSegment').agg({
    'Churn_numeric': ['count', 'sum', 'mean'],
    'MonthlyCharges': 'mean',
    'tenure': 'mean'
}).round(2)

segmento_valor.columns = ['Total_Clientes', 'Churn_Count', 'Taxa_Churn', 
                         'Cobranca_Mensal_Media', 'Tenure_Medio']
segmento_valor['Taxa_Churn_Pct'] = segmento_valor['Taxa_Churn'] * 100

segmento_valor.to_csv('dados/gold/churn_por_segmento.csv')
print("   ✅ churn_por_segmento.csv")

# Criar relatório final
relatorio_final = f"""
RELATÓRIO FINAL - ANÁLISE DE CHURN TELECOM
Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
==========================================
RESUMO EXECUTIVO
==========================================
Total de clientes analisados: {len(df):,}
Taxa geral de churn: {(df['Churn'] == 'Yes').mean()*100:.2f}%
Clientes que cancelaram: {(df['Churn'] == 'Yes').sum():,}
Clientes ativos: {(df['Churn'] == 'No').sum():,}

PRINCIPAIS DESCOBERTAS
==========================================
1. TIPO DE CONTRATO É O FATOR MAIS CRÍTICO:
   - Contratos mensais: {churn_por_contrato.loc['Month-to-month', 'Taxa_Churn_Pct']:.1f}% de churn
   - Contratos anuais: {churn_por_contrato.loc['One year', 'Taxa_Churn_Pct']:.1f}% de churn  
   - Contratos bienais: {churn_por_contrato.loc['Two year', 'Taxa_Churn_Pct']:.1f}% de churn

2. TEMPO DE PERMANÊNCIA IMPACTA SIGNIFICATIVAMENTE:
   - Clientes novos (0-12m): Alto risco
   - Clientes antigos (37m+): Baixo risco
   - Tempo médio de permanência: {df['tenure'].mean():.1f} meses

3. PERFIL DE ALTO RISCO:
   - Contrato mensal + tenure <= 12 meses
   - Cobrança mensal média: R$ {alto_risco['MonthlyCharges'].mean():.2f}
   - Taxa de churn: {(alto_risco['Churn'] == 'Yes').mean()*100:.1f}%

RECOMENDAÇÕES ESTRATÉGICAS
==========================================
1. MIGRAÇÃO PARA CONTRATOS LONGOS:
   - Oferecer desconto de 15-20% em contratos anuais
   - Criar programa de fidelidade para contratos bienais

2. ONBOARDING INTENSIVO:
   - Acompanhamento especial nos primeiros 90 dias
   - Suporte proativo para clientes novos

3. BUNDLING DE SERVIÇOS:
   - Pacotes com 4+ serviços para aumentar retenção
   - Descontos progressivos por número de serviços

4. PROGRAMA DE SEGURANÇA:
   - Trial gratuito de 30 dias para serviços de segurança
   - Educação sobre benefícios de proteção

MÉTRICAS DE SUCESSO
==========================================
- Reduzir taxa de churn de contratos mensais em 10%
- Aumentar conversão para contratos anuais em 25%
- Melhorar retenção de clientes novos em 15%
- Aumentar número médio de serviços por cliente

==========================================
Pipeline de Big Data - Fundamentos de Big Data
Equipe: [NOMES DOS MEMBROS]
Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
==========================================
"""

with open('dados/gold/relatorio_final.txt', 'w', encoding='utf-8') as f:
    f.write(relatorio_final)

print("   ✅ relatorio_final.txt")

print(f"\n" + "="*60)
print("✅ ANÁLISE COMPLETA - Pipeline de Big Data Finalizado!")
print("="*60)

print(f"\n📁 Arquivos gerados na camada Gold:")
print(f"   📊 metricas_por_contrato.csv")
print(f"   📊 perfil_alto_risco.csv") 
print(f"   📊 churn_por_segmento.csv")
print(f"   📄 relatorio_final.txt")
print(f"   🖼️ viz_01_distribuicao_churn.png")
print(f"   🖼️ viz_02_churn_por_contrato.png")
print(f"   🖼️ viz_03_analise_tenure.png")

print(f"\n🎯 PRÓXIMOS PASSOS (AV2):")
print(f"   - Implementar modelo de Machine Learning preditivo")
print(f"   - Criar dashboard interativo")
print(f"   - Desenvolver sistema de alertas automatizados")
print(f"   - Implementar A/B testing de estratégias de retenção")

print(f"\n🏆 PROJETO AV1 CONCLUÍDO COM SUCESSO!")
